# Generators & Comprehensions - Part 3

This will cover more advanced generators and comprehensions (and other iterator) topics, but it starts with some review (and tips).

In [1]:
# Just mapping (no filtering).
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [2]:
# Mapping and filtering.
[x**2 for x in range(10) if x % 3 != 0]

[1, 4, 16, 25, 49, 64]

In [3]:
# Filtering. (Technically also mapping.)
[x for x in range(10) if x % 3 != 0]

[1, 2, 4, 5, 7, 8]

In [4]:
s = ['hello', 'my', 'name', 'is', 'David', 'Vassallo']

In [5]:
# A list of the all-lowercase words in s.
# Only filters.
[word for word in s if word.islower()]

['hello', 'my', 'name', 'is']

In [6]:
# A list of the lengths of all the words in s.
# Only maps.
[len(word) for word in s]

[5, 2, 4, 2, 5, 8]

In [7]:
# A list of the lengths of the all-lowercase words in s
# Maps and filters. 
[len(word) for word in s if word.islower()]

[5, 2, 4, 2]

In [8]:
# A list of the lengths of the all-lowercase words in s, using map and filter.
# This does not use any comprehensions of any kind.
list(map(len, filter(str.islower, s)))

[5, 2, 4, 2]

In [9]:
# Use LBYL to show that map and filter are iterator types.
# Then likewise show, with instances of them, that they are iterator objects.
from collections.abc import Iterator, Mapping

In [10]:
issubclass(map, Iterator)

True

In [11]:
issubclass(filter, Iterator)

True

In [12]:
isinstance(map(len, s), Iterator)

True

In [13]:
isinstance(filter(str.islower, s), Iterator)

True

In [14]:
# The builtin map type should not be confused with mappings (i.e., dict-like objects).
# Show with LBYL that map is not a mapping type.
issubclass(map, Mapping)

False

**Tip 1:** Iterators are iterable. That is, you can call `next` on iterators, but you can also call `iter` on an iterator, which gives you back an equivalent iterator, almost always the very same iterator object.

**Tip 2:** Do not confuse callables with iterators. A callable gives a value when called; an iterator gives a value when passed to `next`.

**Tip 3:** Strongly prefer high-level constructs like `for` over explicitly calling `iter` or `next`.

It is fairly rare to call `next` outside of code that performs general iterator operations such as those performed by the code in `itertools`.

You shouldn't be afraid to use the `iter` and `next` builtin (in any context). But every time you use them, you should be able to clearly articulate why you need or want to use them instead of higher level constructs like `for`.

**Tip 4:** Anytime you write something like:

```python
(x for x in some_expression)
```

Or:

```python
((x, y) for x, y in some_expression)
```

Make sure you understand why you are writing that instead of just:

```python
some_expression
```

Only in rare cases should you write comprehensions of that form.

**Tip 5:** Remember the `sum` builtin.

**Tip 6:** Materialize an iterable when *both*:

1. it might be consumed by iteration (in almost all cases, this is when it's an iterator) *and*

2. you need, or may need, to iterate through its values multiple times.

You should not usually perform materialization unless both of these conditions hold.

Note also that `a = list(b)` and `a = tuple(b)` materialize `b`, but `a = b` does not, because assignment just copies a reference. There is no situation in which running `a = b` and then iterating through `a` behaves any differently from just iterating through `b`.



In [15]:
import inspect
import itertools

In [16]:
print(inspect.getdoc(itertools))

Functional tools for creating and using iterators.

Infinite iterators:
count(start=0, step=1) --> start, start+step, start+2*step, ...
cycle(p) --> p0, p1, ... plast, p0, p1, ...
repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times

Iterators terminating on the shortest input sequence:
accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ...
chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ...
compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
filterfalse(pred, seq) --> elements of seq where pred(elem) is False
islice(seq, [start,] stop [, step]) --> elements from
       seq[start:stop:step]
pairwise(s) --> (s[0],s[1]), (s[1],s[2]), (s[2], s[3]), ...
starmap(fun, seq) --> fun(*seq[0]), fun(*seq[1]), ...
tee(it, n=2) --> (it1, it2 , ... itn) splits one it

In [17]:
help(inspect.getgeneratorstate)

Help on function getgeneratorstate in module inspect:

getgeneratorstate(generator)
    Get current state of a generator-iterator.
    
    Possible states are:
      GEN_CREATED: Waiting to start execution.
      GEN_RUNNING: Currently being executed by the interpreter.
      GEN_SUSPENDED: Currently suspended at a yield expression.
      GEN_CLOSED: Execution has completed.



## Generator states

In [18]:
def gen1(): 
    yield 'a'
    yield 'b'
    yield 'c'

In [19]:
g = gen1()

In [20]:
inspect.getgeneratorstate(g)

'GEN_CREATED'

In [21]:
next(g)

'a'

In [22]:
inspect.getgeneratorstate(g)

'GEN_SUSPENDED'

In [23]:
next(g)

'b'

In [24]:
inspect.getgeneratorstate(g)

'GEN_SUSPENDED'

In [25]:
next(g)

'c'

In [26]:
inspect.getgeneratorstate(g)

'GEN_SUSPENDED'

In [27]:
next(g)

StopIteration: 

In [28]:
inspect.getgeneratorstate(g)

'GEN_CLOSED'

In [29]:
c = gen1()

In [30]:
inspect.getgeneratorstate(c)

'GEN_CREATED'

In [31]:
c.close()

In [32]:
inspect.getgeneratorstate(c)

'GEN_CLOSED'

In [33]:
next(c)

StopIteration: 

In [34]:
squares = (x**2 for x in itertools.count(start=1))

In [35]:
inspect.getgeneratorstate(squares)

'GEN_CREATED'

In [36]:
next(squares)

1

In [37]:
inspect.getgeneratorstate(squares)

'GEN_SUSPENDED'

In [38]:
squares.close()

In [39]:
inspect.getgeneratorstate(squares)

'GEN_CLOSED'

In [40]:
next(squares)

StopIteration: 

## `finally` blocks in generators

In [41]:
def gen(): 
    try: 
        yield 1
        yield 2
        yield 3
    finally: 
        print('Done.')

In [42]:
[x for x in gen()] # don't do this

Done.


[1, 2, 3]

In [43]:
list(gen()) # do this instead

Done.


[1, 2, 3]

In [44]:
def f(): 
    gen()

In [45]:
f()

In [46]:
def f2(): 
    g = gen()
    return next(g)

In [47]:
f2()

Done.


1

## `GeneratorExit` is raised when destroying suspended generators

In [48]:
print(GeneratorExit.__doc__)

Request that a generator exit.


In [49]:
GeneratorExit.__bases__

(BaseException,)

In [50]:
ValueError.__bases__

(Exception,)

In [51]:
TypeError.__bases__

(Exception,)

In [52]:
StopIteration.__bases__

(Exception,)

In [53]:
SystemExit.__bases__

(BaseException,)

In [54]:
Exception.__bases__

(BaseException,)

In [55]:
def gen2(): 
    try: 
        yield 1
        yield 2
        yield 3
    except GeneratorExit: 
        print('Interrupted.')
    finally: 
        print('Done.')

In [56]:
list(gen2())

Done.


[1, 2, 3]

In [57]:
def f3(): 
    g = gen2()
    return next(g)

In [58]:
f3()

Interrupted.
Done.


1

## It is an error to yield after `GeneratorExit`

In [59]:
def gen_bad(): 
    try: 
        yield 1
        yield 2
        yield 3
    except GeneratorExit: 
        print('Interrupted.')
        yield 4
    finally: 
        print('Done.')

In [60]:
def f4(): 
    g = gen_bad()
    return next(g)

In [61]:
f4()

Exception ignored in: <generator object gen_bad at 0x00000152A3EF9620>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_16292\1205670228.py", line 1, in <cell line: 1>
RuntimeError: generator ignored GeneratorExit


Interrupted.


1

In [62]:
# s = [1]
# s.append(s)

## Generator objects with reference cycles

In [63]:
def gen_rcycle():     
    try: 
        s = [1]
        yield s
    finally: 
        print('Done')

In [64]:
def do_stuff(): 
    ob = gen_rcycle() 
    next(ob).append(ob)

In [65]:
do_stuff()

In [66]:
import gc

In [67]:
gc.collect()

Done


1382

## Generator objects are not context managers

In [68]:
def do_more_stuff(): 
    ob = gen_rcycle() 
    next(ob).append(ob)
    return ob

In [69]:
with do_more_stuff(): 
    print('Use resource acquired')

Done


AttributeError: __enter__

In [70]:
gc.collect()

465

## Generator objects do have a `close` method

In [71]:
ob1 = do_more_stuff()

In [72]:
type(ob1)

generator

In [73]:
dir(ob1)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [74]:
help(ob1.close)

Help on built-in function close:

close(...) method of builtins.generator instance
    close() -> raise GeneratorExit inside generator.



In [75]:
ob1 = do_more_stuff()
try: 
    print('If I use ob1, I would use it here')
finally: 
    ob1.close()

If I use ob1, I would use it here
Done


In [76]:
import contextlib

In [77]:
with contextlib.closing(do_more_stuff()): 
    print('Use resource acquired')

Use resource acquired
Done


**TODO:** Examine materialization and acquision/release in `itertools.tee`.

## Review of materialization

In [78]:
def product_three(one, two, three): 
    my_one = list(one)
    my_two = list(two)
    my_three = list(three)
    return ((x, y, z) for x in my_one for y in my_two for z in my_three)

In [79]:
groups = product_three(['a', 'b', 'c'], (x for x in range(4)), (100, 200, 300))

In [80]:
list(groups)

Done


[('a', 0, 100),
 ('a', 0, 200),
 ('a', 0, 300),
 ('a', 1, 100),
 ('a', 1, 200),
 ('a', 1, 300),
 ('a', 2, 100),
 ('a', 2, 200),
 ('a', 2, 300),
 ('a', 3, 100),
 ('a', 3, 200),
 ('a', 3, 300),
 ('b', 0, 100),
 ('b', 0, 200),
 ('b', 0, 300),
 ('b', 1, 100),
 ('b', 1, 200),
 ('b', 1, 300),
 ('b', 2, 100),
 ('b', 2, 200),
 ('b', 2, 300),
 ('b', 3, 100),
 ('b', 3, 200),
 ('b', 3, 300),
 ('c', 0, 100),
 ('c', 0, 200),
 ('c', 0, 300),
 ('c', 1, 100),
 ('c', 1, 200),
 ('c', 1, 300),
 ('c', 2, 100),
 ('c', 2, 200),
 ('c', 2, 300),
 ('c', 3, 100),
 ('c', 3, 200),
 ('c', 3, 300)]

## Spilling the `tee`: the mystery of 57

In [81]:
help(itertools.tee)

Help on built-in function tee in module itertools:

tee(iterable, n=2, /)
    Returns a tuple of n independent iterators.



In [82]:
s = [1, 2, 3, 4]

In [83]:
i1, i2 = itertools.tee(s)

In [84]:
next(i1)

1

In [85]:
next(i1)

2

In [86]:
next(i2)

1

In [87]:
next(i2)

2

In [88]:
list(i2)

[3, 4]

In [89]:
list(i1)

[3, 4]

In [90]:
squares = (x**2 for x in s)

In [91]:
s1, s2 = itertools.tee(squares)

In [92]:
list(s1)

[1, 4, 9, 16]

In [93]:
list(s2)

[1, 4, 9, 16]

In [94]:
inspect.getgeneratorstate(squares)

'GEN_CLOSED'

In [95]:
next(squares)

StopIteration: 

In [96]:
squares = (x**2 for x in itertools.count(start=1))

In [97]:
s1, s2 = itertools.tee(squares)

In [98]:
next(s1)

1

In [99]:
next(s1)

4

In [100]:
next(s1)

9

In [101]:
next(s1)

16

In [102]:
next(s1)

25

In [103]:
next(s2)

1

In [104]:
next(s2)

4

In [105]:
next(s2)

9

In [106]:
next(s2)

16

In [107]:
inspect.getgeneratorstate(squares)

'GEN_SUSPENDED'

In [108]:
squares.close()

In [109]:
next(s2)

25

In [110]:
next(s2)

StopIteration: 

In [111]:
next(s1)

StopIteration: 

In [112]:
x, y = itertools.tee(itertools.count())

In [113]:
next(y)

0

In [114]:
next(y)

1

In [115]:
next(y)

2

In [116]:
z = zip(x, y)

In [117]:
next(z)

(0, 3)

In [118]:
next(z)

(1, 4)

In [119]:
next(z)

(2, 5)

In [120]:
# for _ in range(100_000_000): 
#    next(z)

In [121]:
next(z)

(3, 6)

In [122]:
# Takes a long time, but still doesn't use much memory:

# for _ in range(1_000_000_000): 
#     next(z)

In [123]:
class TeeInspector: 
    
    def __init__(self, val): 
        self.val = val
        
    def __repr__(self):
        return f'{type(self).__name__}({self.val!r})'
    
    def __del__(self): 
        print(f'#{self.val} is being finalized.')

In [124]:
t = TeeInspector(1)

In [125]:
del t

#1 is being finalized.


In [126]:
g = (TeeInspector(x) for x in range(10))

In [127]:
next(g).val

#0 is being finalized.


0

In [128]:
next(g).val

#1 is being finalized.


1

In [129]:
for x in g: 
    print(x.val)

2
#2 is being finalized.
3
#3 is being finalized.
4
#4 is being finalized.
5
#5 is being finalized.
6
#6 is being finalized.
7
#7 is being finalized.
8
#8 is being finalized.
9


In [130]:
x.val

9

In [131]:
del x

#9 is being finalized.


In [132]:
g = (TeeInspector(x) for x in range(7))

In [133]:
x, y = itertools.tee(g)

In [134]:
next(y).val

0

In [135]:
next(y).val

1

In [136]:
next(y).val

2

In [137]:
next(x).val

0

In [138]:
next(x).val

1

In [139]:
next(y).val

3

In [140]:
next(y).val

4

In [141]:
next(y).val

5

In [142]:
next(x).val

2

In [143]:
next(x).val

3

In [144]:
next(x).val

4

In [145]:
next(x).val

5

In [146]:
next(x).val

6

In [147]:
next(x).val

StopIteration: 

In [148]:
next(y).val

6

In [149]:
next(y).val

StopIteration: 

In [150]:
del x, y

#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.


In [151]:
big = (TeeInspector(x) for x in range(200))

In [152]:
x, y = itertools.tee(big)

In [153]:
for _ in range(100): 
    b = next(y).val

In [154]:
for _ in range(50): 
    c = next(x).val

In [155]:
for _ in range(50): 
    c = next(x).val

#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.
#7 is being finalized.
#8 is being finalized.
#9 is being finalized.
#10 is being finalized.
#11 is being finalized.
#12 is being finalized.
#13 is being finalized.
#14 is being finalized.
#15 is being finalized.
#16 is being finalized.
#17 is being finalized.
#18 is being finalized.
#19 is being finalized.
#20 is being finalized.
#21 is being finalized.
#22 is being finalized.
#23 is being finalized.
#24 is being finalized.
#25 is being finalized.
#26 is being finalized.
#27 is being finalized.
#28 is being finalized.
#29 is being finalized.
#30 is being finalized.
#31 is being finalized.
#32 is being finalized.
#33 is being finalized.
#34 is being finalized.
#35 is being finalized.
#36 is being finalized.
#37 is being finalized.
#38 is being finalized.
#39 is being finalized.
#40 is being finalized.
#41 is being finalized.
#4

In [156]:
del x, y

#57 is being finalized.
#58 is being finalized.
#59 is being finalized.
#60 is being finalized.
#61 is being finalized.
#62 is being finalized.
#63 is being finalized.
#64 is being finalized.
#65 is being finalized.
#66 is being finalized.
#67 is being finalized.
#68 is being finalized.
#69 is being finalized.
#70 is being finalized.
#71 is being finalized.
#72 is being finalized.
#73 is being finalized.
#74 is being finalized.
#75 is being finalized.
#76 is being finalized.
#77 is being finalized.
#78 is being finalized.
#79 is being finalized.
#80 is being finalized.
#81 is being finalized.
#82 is being finalized.
#83 is being finalized.
#84 is being finalized.
#85 is being finalized.
#86 is being finalized.
#87 is being finalized.
#88 is being finalized.
#89 is being finalized.
#90 is being finalized.
#91 is being finalized.
#92 is being finalized.
#93 is being finalized.
#94 is being finalized.
#95 is being finalized.
#96 is being finalized.
#97 is being finalized.
#98 is being fin

In [157]:
big = (TeeInspector(x) for x in range(200))

In [158]:
x, y = itertools.tee(big)

In [159]:
for _ in range(100): 
    next(y)

In [160]:
for _ in range(100):
    print(next(x).val)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.
#7 is being finalized.
#8 is being finalized.
#9 is being finalized.
#10 is being finalized.
#11 is being finalized.
#12 is being finalized.
#13 is being finalized.
#14 is being finalized.
#15 is being finalized.
#16 is being finalized.
#17 is being finalized.
#18 is being finalized.
#19 is being finalized.
#20 is being finalized.
#21 is being finalized.
#22 is being finalized.
#23 is being finalized.
#24 is being finalized.
#25 is being finalized.
#26 is being finalized.
#27 is being finalized.
#28 is being finalized.
#29 is being finalized.
#30 is being finalized.
#31 is being finalized.
#32 is being finalized.
#33 is being finalized.
#34 is being finalized.
#35 is be

In [161]:
for _ in range(100):
    next(y)

In [162]:
for _ in range(100):
    print(next(x).val)

100
101
102
103
104
105
106
107
108
109
110
111
112
113
#57 is being finalized.
#58 is being finalized.
#59 is being finalized.
#60 is being finalized.
#61 is being finalized.
#62 is being finalized.
#63 is being finalized.
#64 is being finalized.
#65 is being finalized.
#66 is being finalized.
#67 is being finalized.
#68 is being finalized.
#69 is being finalized.
#70 is being finalized.
#71 is being finalized.
#72 is being finalized.
#73 is being finalized.
#74 is being finalized.
#75 is being finalized.
#76 is being finalized.
#77 is being finalized.
#78 is being finalized.
#79 is being finalized.
#80 is being finalized.
#81 is being finalized.
#82 is being finalized.
#83 is being finalized.
#84 is being finalized.
#85 is being finalized.
#86 is being finalized.
#87 is being finalized.
#88 is being finalized.
#89 is being finalized.
#90 is being finalized.
#91 is being finalized.
#92 is being finalized.
#93 is being finalized.
#94 is being finalized.
#95 is being finalized.
#96 is b

In [164]:
two_hundred_inspectors = (TeeInspector(x) for x in range(200))

In [165]:
back, front = itertools.tee(two_hundred_inspectors)

In [166]:
for _ in range(100): 
    next(front)

In [167]:
for _ in range(5): 
    next(back)

In [168]:
del back

#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.
#7 is being finalized.
#8 is being finalized.
#9 is being finalized.
#10 is being finalized.
#11 is being finalized.
#12 is being finalized.
#13 is being finalized.
#14 is being finalized.
#15 is being finalized.
#16 is being finalized.
#17 is being finalized.
#18 is being finalized.
#19 is being finalized.
#20 is being finalized.
#21 is being finalized.
#22 is being finalized.
#23 is being finalized.
#24 is being finalized.
#25 is being finalized.
#26 is being finalized.
#27 is being finalized.
#28 is being finalized.
#29 is being finalized.
#30 is being finalized.
#31 is being finalized.
#32 is being finalized.
#33 is being finalized.
#34 is being finalized.
#35 is being finalized.
#36 is being finalized.
#37 is being finalized.
#38 is being finalized.
#39 is being finalized.
#40 is being finalized.
#41 is being finalized.
#4

In [169]:
two_hundred_inspectors = (TeeInspector(x) for x in range(200))

In [170]:
back, front = itertools.tee(two_hundred_inspectors)

#57 is being finalized.
#58 is being finalized.
#59 is being finalized.
#60 is being finalized.
#61 is being finalized.
#62 is being finalized.
#63 is being finalized.
#64 is being finalized.
#65 is being finalized.
#66 is being finalized.
#67 is being finalized.
#68 is being finalized.
#69 is being finalized.
#70 is being finalized.
#71 is being finalized.
#72 is being finalized.
#73 is being finalized.
#74 is being finalized.
#75 is being finalized.
#76 is being finalized.
#77 is being finalized.
#78 is being finalized.
#79 is being finalized.
#80 is being finalized.
#81 is being finalized.
#82 is being finalized.
#83 is being finalized.
#84 is being finalized.
#85 is being finalized.
#86 is being finalized.
#87 is being finalized.
#88 is being finalized.
#89 is being finalized.
#90 is being finalized.
#91 is being finalized.
#92 is being finalized.
#93 is being finalized.
#94 is being finalized.
#95 is being finalized.
#96 is being finalized.
#97 is being finalized.
#98 is being fin

In [171]:
for _ in range(30): 
    next(front)

In [172]:
for _ in range(30): 
    next(back)

In [173]:
for _ in range(70): 
    next(back)

In [174]:
for _ in range(70): 
    next(front)

#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.
#7 is being finalized.
#8 is being finalized.
#9 is being finalized.
#10 is being finalized.
#11 is being finalized.
#12 is being finalized.
#13 is being finalized.
#14 is being finalized.
#15 is being finalized.
#16 is being finalized.
#17 is being finalized.
#18 is being finalized.
#19 is being finalized.
#20 is being finalized.
#21 is being finalized.
#22 is being finalized.
#23 is being finalized.
#24 is being finalized.
#25 is being finalized.
#26 is being finalized.
#27 is being finalized.
#28 is being finalized.
#29 is being finalized.
#30 is being finalized.
#31 is being finalized.
#32 is being finalized.
#33 is being finalized.
#34 is being finalized.
#35 is being finalized.
#36 is being finalized.
#37 is being finalized.
#38 is being finalized.
#39 is being finalized.
#40 is being finalized.
#41 is being finalized.
#4

In [175]:
del x, y, back, front, two_hundred_inspectors

In [176]:
two_hundred_inspectors = (TeeInspector(x) for x in range(200))

In [177]:
one, two, three = itertools.tee(two_hundred_inspectors, 3)

In [178]:
for _ in range(100): 
    next(one)

In [179]:
for _ in range(100):
    next(two)

In [180]:
for _ in range(100): 
    next(three)

#0 is being finalized.
#1 is being finalized.
#2 is being finalized.
#3 is being finalized.
#4 is being finalized.
#5 is being finalized.
#6 is being finalized.
#7 is being finalized.
#8 is being finalized.
#9 is being finalized.
#10 is being finalized.
#11 is being finalized.
#12 is being finalized.
#13 is being finalized.
#14 is being finalized.
#15 is being finalized.
#16 is being finalized.
#17 is being finalized.
#18 is being finalized.
#19 is being finalized.
#20 is being finalized.
#21 is being finalized.
#22 is being finalized.
#23 is being finalized.
#24 is being finalized.
#25 is being finalized.
#26 is being finalized.
#27 is being finalized.
#28 is being finalized.
#29 is being finalized.
#30 is being finalized.
#31 is being finalized.
#32 is being finalized.
#33 is being finalized.
#34 is being finalized.
#35 is being finalized.
#36 is being finalized.
#37 is being finalized.
#38 is being finalized.
#39 is being finalized.
#40 is being finalized.
#41 is being finalized.
#4

In [181]:
help(itertools.pairwise)

Help on class pairwise in module itertools:

class pairwise(builtins.object)
 |  pairwise(iterable, /)
 |  
 |  Return an iterator of overlapping pairs taken from the input iterator.
 |  
 |  s -> (s0,s1), (s1,s2), (s2, s3), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.

